# Lesson 2: Langchain Expression language

In [ ]:
import os
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')

# Simple Chain

In [ ]:
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
model = ChatOpenAI(api_key=api_key)
output_parser = StrOutputParser()

chain = prompt | model | output_parser
chain.invoke({"topic": "bears"})

# More complex chain

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

retriever.get_relevant_documents("where did harrison work?")

In [ ]:
retriever.get_relevant_documents("what do bears like to eat?")

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question:{question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain.schema.runnable import RunnableMap

chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

chain.invoke({"question": "where did harrison work?"})

In [ ]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})
inputs.invoke({"question": "where did harrison work?"})

# Bind

In [ ]:
functions = [
    {
        "name": "weather_search",
        "description": "Search for weather given an airport code",
        "parameters": {
            "type": "object",
            "properties": {
                "airport_code": {
                    "type": "string",
                    "description": "The airport code to get the weather for"
                },
            },
            "required": ["airport_code"]
        }
    }
]

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)

model = ChatOpenAI(temperature=0).bind(functions=functions)

In [ ]:
runnable = prompt | model

runnable.invoke({"input": "what is the weather in sf"})

In [ ]:
functions = [
    {
        "name": "weather_search",
        "description": "Search for weather given an airport code",
        "parameters": {
            "type": "object",
            "properties": {
                "airport_code": {
                    "type": "string",
                    "description": "The airport code to get the weather for"
                },
            },
            "required": ["airport_code"]
        }
    },
    {
        "name": "sports_search",
        "description": "Search for news of recent sport events",
        "parameters": {
            "type": "object",
            "properties": {
                "team_name": {
                    "type": "string",
                    "description": "The sports team to search for"
                },
            },
            "required": ["team_name"]
        }
    }
]

In [ ]:
model = model.bind(functions=functions)

runnable = prompt | model
runnable.invoke({"input": "how did the patriots do yesterday?"})

# Fallbacks

In [ ]:
from langchain_openai.llms import OpenAI
import json

simple_model = OpenAI(
    temperature=0,
    max_tokens=1000,
    model="gpt-3.5-turbo-instruct"
)

In [ ]:
def ensure_valid_json(response):
    try:
        parsed_response = json.loads(response)
        return parsed_response
    except json.JSONDecodeError:
        return {"error": "Failed to decode JSON"}


simple_chain = simple_model | ensure_valid_json

challenge = """write three poems in a JSON blob, where each poem is a JSON blob with keys 'title', 'author', and 'first_line'.
Output the entire response in a valid JSON format.
"""

simple_chain.invoke(challenge)

In [ ]:
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

In [ ]:
chain.invoke(challenge)

In [ ]:
final_chain = simple_chain.with_fallbacks([chain])

In [ ]:
final_chain.invoke(challenge)

# Interface

In [ ]:
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
model = ChatOpenAI(api_key=api_key)
output_parser = StrOutputParser()

chain = prompt | model | output_parser
chain.invoke({"topic": "bears"})

In [ ]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

for t in chain.stream({"topic": "bears"}):
    print(t)


In [ ]:
response = await chain.ainvoke({"topic": "bears"})
response